# Revisão de tópicos para a AF

## Modelos lineares com regularização

#### Modelo

Em todos os modelos de regressão linear, o modelo é o mesmo:

$$
\hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \cdots + \theta_n x_n
$$

A diferença entre as versões abaixo está na <span style='color:red'>função de perda</span>.

#### Mean squared-error

A função básica de perda da regressão linear é o erro quadrático médio (*mean squared error* - MSE).

Para um conjunto de treino de $m$ exemplos, seja $x^{(i)}$ o vetor de *features* do $i$-ésimo exemplo, $y^{(i)}$ o *target* do $i$-ésimo exemplo e $\hat{y}^{(i)}$ o valor predito para o $i$-ésimo exemplo. O valor do MSE é dado por:

$$
\text{MSE}(\overrightarrow{\theta}) = 
\sum_{i = 1}^{m} \left( \hat{y}^{(i)} - y^{(i)} \right)^{2}
$$

#### Alguns dados de exemplo

In [1]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

X, y = load_diabetes(return_X_y=True, scaled=False)

poly = PolynomialFeatures(degree=2, include_bias=False)
X = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

### Sem regularização

#### Função de perda

A função de perda sem regularização é simplesmente o MSE:

$$
\mathcal{L}(\overrightarrow{\theta}) = \frac{1}{2}\text{MSE}(\overrightarrow{\theta})
$$

#### Coisas importantes

- Não requer normalização
- Não tem hiperparâmetros
- Não pode ter colinearidade de colunas
- Não combate overfitting

#### Aplicação

- Interpretabilidade: os coeficientes representam "aumento do target por aumento unitário da feature"

#### Exemplo em sklearn

In [2]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [3]:
from sklearn.metrics import mean_squared_error

y_pred_train = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
print(f'Mean Squared Error, train: {mse_train=:.2f}')

Mean Squared Error, train: mse_train=2393.14


In [4]:
y_pred_test = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)
print(f'Mean Squared Error, test: {mse_test=:.2f}')

Mean Squared Error, test: mse_test=3096.03


In [5]:
print(f'Model coefficients:')
for i, coef in enumerate(model.coef_):
    print(f'  {i+1}: {coef.round(2)}')
print(f'Model intercept: {model.intercept_.round(2)}')

Model coefficients:
  1: -9.59
  2: -11.24
  3: -19.7
  4: 10.64
  5: -16.59
  6: 26.14
  7: 11.13
  8: -160.29
  9: -1768.67
  10: -13.82
  11: 0.03
  12: 0.53
  13: -0.05
  14: 0.01
  15: -0.0
  16: -0.04
  17: 0.08
  18: 1.31
  19: 0.41
  20: 0.01
  21: -33.71
  22: 0.36
  23: 0.41
  24: 0.21
  25: 0.25
  26: -1.24
  27: -26.74
  28: 18.52
  29: 0.27
  30: 0.09
  31: 0.12
  32: -0.09
  33: 0.12
  34: -0.01
  35: -1.43
  36: 3.63
  37: 0.02
  38: -0.0
  39: 0.09
  40: -0.08
  41: -0.11
  42: -0.18
  43: -2.48
  44: -0.05
  45: 0.18
  46: -0.28
  47: -0.31
  48: -1.28
  49: -5.05
  50: -0.04
  51: 0.11
  52: 0.2
  53: 0.37
  54: 3.38
  55: 0.03
  56: 0.15
  57: 2.2
  58: 4.46
  59: 0.12
  60: 16.31
  61: 15.19
  62: 0.86
  63: 273.67
  64: 1.88
  65: 0.02
Model intercept: 3792.08


### Ridge

#### Função de perda

A função de perda da regressão Ridge (ou regularização $L_2$) incorpora um termo de penalidade para a soma dos quadrados dos coeficientes da regressão. O hiperparâmetro $\alpha$ controla a intensidade da penalidade.

$$
\mathcal{L} = \frac{1}{2}\text{MSE}(\overrightarrow{\theta}) + \frac{\alpha}{2} \sum_{i=1}^{n} \theta_i^2
$$

#### Coisas importantes

- Requer normalização dos dados
- Valores mais altos de $\alpha$ combatem colinearidade
- Interpretação mais difícil: a escolha de $\alpha$ impacta os valores de $\theta$, e portanto mudam a interpretação

#### Aplicação

- Combate colinearidade
- Combate overfitting

#### Exemplo em sklearn

In [6]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

- Pouca regularização:

In [7]:
alpha = 1e-3

In [8]:
model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Ridge(alpha=alpha)),
])
model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor', Ridge(alpha=0.001))])

In [9]:
y_pred_train = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
print(f'Mean Squared Error, train, {alpha=}: {mse_train=:.2f}')

Mean Squared Error, train, alpha=0.001: mse_train=2403.47


In [10]:
y_pred_test = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)
print(f'Mean Squared Error, test, {alpha=}: {mse_test=:.2f}')

Mean Squared Error, test, alpha=0.001: mse_test=3068.03


In [11]:
print(f'Model coefficients:')
for i, coef in enumerate(model.named_steps['regressor'].coef_):
    print(f'  {i+1}: {coef.round(2)}')
print(f'Model intercept: {model.named_steps['regressor'].intercept_.round(2)}')

Model coefficients:
  1: -124.43
  2: -27.17
  3: -104.57
  4: 132.7
  5: -370.15
  6: 498.72
  7: 113.2
  8: -93.93
  9: -437.42
  10: -143.06
  11: 37.41
  12: 21.92
  13: -19.43
  14: 12.14
  15: -16.11
  16: -95.03
  17: 77.52
  18: 116.2
  19: 21.2
  20: 20.65
  21: -27.17
  22: 9.06
  23: 22.11
  24: 12.44
  25: 26.1
  26: -26.65
  27: -84.58
  28: 36.61
  29: 14.38
  30: 25.99
  31: 82.25
  32: -260.89
  33: 232.21
  34: 24.56
  35: -64.18
  36: 144.81
  37: 13.54
  38: -7.16
  39: 394.34
  40: -244.61
  41: -122.09
  42: -19.87
  43: -210.51
  44: -97.66
  45: 598.55
  46: -519.85
  47: -16.21
  48: -138.28
  49: -437.22
  50: -107.57
  51: 139.02
  52: -116.02
  53: -119.08
  54: 205.51
  55: 18.68
  56: -38.69
  57: 41.45
  58: 14.97
  59: 113.82
  60: 174.23
  61: -57.85
  62: 133.79
  63: 673.43
  64: 124.32
  65: 48.86
Model intercept: 153.74


- Regularização média

In [12]:
alpha = 1.0

In [13]:
model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Ridge(alpha=alpha)),
])
model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('regressor', Ridge())])

In [14]:
y_pred_train = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
print(f'Mean Squared Error, test, {alpha=}: {mse_train=:.2f}')

Mean Squared Error, test, alpha=1.0: mse_train=2577.87


In [15]:
y_pred_test = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)
print(f'Mean Squared Error, test, {alpha=}: {mse_test=:.2f}')

Mean Squared Error, test, alpha=1.0: mse_test=2638.69


In [16]:
print(f'Model coefficients:')
for i, coef in enumerate(model.named_steps['regressor'].coef_):
    print(f'  {i+1}: {coef.round(2)}')
print(f'Model intercept: {model.named_steps['regressor'].intercept_.round(2)}')

Model coefficients:
  1: -53.33
  2: -20.72
  3: -16.59
  4: -9.08
  5: -24.55
  6: 9.2
  7: -17.18
  8: 11.39
  9: -16.2
  10: -33.97
  11: 26.07
  12: 28.61
  13: -10.8
  14: 11.9
  15: -1.47
  16: -36.91
  17: 15.38
  18: 25.47
  19: 18.29
  20: -3.26
  21: -20.72
  22: 2.6
  23: 18.49
  24: 5.7
  25: -11.63
  26: 1.8
  27: -14.79
  28: -0.31
  29: 2.13
  30: 20.05
  31: 47.9
  32: -23.32
  33: 8.55
  34: -7.12
  35: -1.21
  36: -0.75
  37: 16.0
  38: -8.11
  39: 4.03
  40: 13.05
  41: 6.84
  42: -1.09
  43: 16.06
  44: -41.03
  45: -0.13
  46: 9.29
  47: 14.95
  48: -34.53
  49: -5.27
  50: -2.09
  51: -3.79
  52: -7.94
  53: 4.42
  54: 36.24
  55: 8.84
  56: -12.08
  57: -10.48
  58: 17.5
  59: 6.77
  60: 12.97
  61: -12.31
  62: 22.94
  63: 12.78
  64: 21.12
  65: 25.6
Model intercept: 153.74


- Regularização alta

In [17]:
alpha = 1000.0

In [18]:
model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Ridge(alpha=alpha)),
])
model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor', Ridge(alpha=1000.0))])

In [19]:
y_pred_train = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
print(f'Mean Squared Error, test, {alpha=}: {mse_train=:.2f}')

Mean Squared Error, test, alpha=1000.0: mse_train=3079.07


In [20]:
y_pred_test = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)
print(f'Mean Squared Error, test, {alpha=}: {mse_test=:.2f}')

Mean Squared Error, test, alpha=1000.0: mse_test=2942.16


In [21]:
print(f'Model coefficients:')
for i, coef in enumerate(model.named_steps['regressor'].coef_):
    print(f'  {i+1}: {coef.round(2)}')
print(f'Model intercept: {model.named_steps['regressor'].intercept_.round(2)}')

Model coefficients:
  1: 0.14
  2: -1.16
  3: 3.62
  4: 2.34
  5: -0.42
  6: -0.81
  7: -1.57
  8: 0.99
  9: 2.96
  10: 0.89
  11: 0.35
  12: -0.31
  13: 2.02
  14: 1.34
  15: -0.21
  16: -0.69
  17: -1.01
  18: 0.84
  19: 1.35
  20: 0.58
  21: -1.16
  22: 0.67
  23: 0.08
  24: -1.22
  25: -1.47
  26: -2.25
  27: -0.2
  28: -0.05
  29: -0.64
  30: 3.8
  31: 3.8
  32: 1.76
  33: 1.0
  34: 0.54
  35: 2.24
  36: 3.91
  37: 3.03
  38: 2.41
  39: 1.03
  40: 0.35
  41: -0.17
  42: 1.72
  43: 3.18
  44: 2.03
  45: -0.42
  46: -0.68
  47: -1.28
  48: 0.36
  49: 0.91
  50: 0.15
  51: -0.86
  52: -1.76
  53: 0.06
  54: 0.32
  55: -0.31
  56: -1.35
  57: -0.55
  58: -0.17
  59: -1.23
  60: 0.88
  61: 1.48
  62: 1.17
  63: 2.84
  64: 2.23
  65: 1.02
Model intercept: 153.74


### LASSO

#### Função de perda

A função de perda da regressão LASSO (Least Absolute Shrinking and Selection Operator, também conhecida como regularização $L_1$) incorpora um termo de penalidade para a soma dos valores absolutos dos coeficientes da regressão. O hiperparâmetro $\alpha$ controla a intensidade da penalidade.

$$
\mathcal{L} = \frac{1}{2}\text{MSE}(\overrightarrow{\theta}) + \alpha \sum_{i=1}^{n} |\theta_i|
$$

#### Coisas importantes

- Requer normalização dos dados
- Valores mais altos de $\alpha$ servem para selecionar features
- Interpretação mais difícil: a escolha de $\alpha$ impacta os valores de $\theta$, e portanto mudam a interpretação

#### Aplicação

- Seleciona features
- Combate overfitting

#### Exemplo em sklearn

In [22]:
from sklearn.linear_model import Lasso

- Pouca regularização:

In [23]:
alpha = 1e-3

In [24]:
model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Lasso(alpha=alpha)),
])
model.fit(X_train, y_train)

/home/fjayres/miniconda3/envs/ml/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.362e+05, tolerance: 2.145e+02
  model = cd_fast.enet_coordinate_descent(


Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor', Lasso(alpha=0.001))])

In [25]:
y_pred_train = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
print(f'Mean Squared Error, train, {alpha=}: {mse_train=:.2f}')

Mean Squared Error, train, alpha=0.001: mse_train=2505.28


In [26]:
y_pred_test = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)
print(f'Mean Squared Error, test, {alpha=}: {mse_test=:.2f}')

Mean Squared Error, test, alpha=0.001: mse_test=2699.03


In [27]:
print(f'Model coefficients:')
for i, coef in enumerate(model.named_steps['regressor'].coef_):
    print(f'  {i+1}: {coef.round(2)}')
print(f'Model intercept: {model.named_steps['regressor'].intercept_.round(2)}')

Model coefficients:
  1: -133.26
  2: -7.52
  3: -55.03
  4: 19.19
  5: -70.5
  6: 41.18
  7: -7.9
  8: 15.06
  9: -21.61
  10: -58.36
  11: 30.84
  12: 31.48
  13: -21.44
  14: 32.45
  15: -12.25
  16: -79.79
  17: 79.74
  18: 100.98
  19: 28.22
  20: 2.85
  21: -47.39
  22: 8.2
  23: 26.78
  24: 21.37
  25: -10.81
  26: -7.69
  27: -39.61
  28: -2.35
  29: 15.09
  30: 28.31
  31: 97.14
  32: -50.62
  33: 40.0
  34: -6.49
  35: -25.69
  36: 12.75
  37: 20.59
  38: -25.17
  39: 29.16
  40: 14.9
  41: -14.23
  42: -31.51
  43: -1.81
  44: -88.94
  45: -12.43
  46: 25.93
  47: 53.37
  48: -106.34
  49: 19.46
  50: 37.99
  51: -7.34
  52: -31.49
  53: 31.1
  54: 56.7
  55: -37.72
  56: -46.44
  57: -9.07
  58: -14.38
  59: 35.55
  60: 32.01
  61: -33.07
  62: 64.4
  63: 28.04
  64: 17.73
  65: 39.51
Model intercept: 153.74


- Regularização média

In [28]:
alpha = 1.0

In [29]:
model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Lasso(alpha=alpha)),
])
model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('regressor', Lasso())])

In [30]:
y_pred_train = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
print(f'Mean Squared Error, test, {alpha=}: {mse_train=:.2f}')

Mean Squared Error, test, alpha=1.0: mse_train=2859.93


In [31]:
y_pred_test = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)
print(f'Mean Squared Error, test, {alpha=}: {mse_test=:.2f}')

Mean Squared Error, test, alpha=1.0: mse_test=2833.57


In [32]:
print(f'Model coefficients:')
for i, coef in enumerate(model.named_steps['regressor'].coef_):
    print(f'  {i+1}: {coef.round(2)}')
print(f'Model intercept: {model.named_steps['regressor'].intercept_.round(2)}')

Model coefficients:
  1: 0.0
  2: -0.0
  3: 0.0
  4: 0.0
  5: -0.0
  6: -0.0
  7: -0.0
  8: 0.0
  9: 18.36
  10: 0.0
  11: 2.12
  12: 0.0
  13: 0.0
  14: 0.0
  15: -0.0
  16: -0.0
  17: -0.0
  18: 0.0
  19: 0.0
  20: 0.0
  21: -0.0
  22: -0.0
  23: 0.0
  24: -4.21
  25: -0.0
  26: -6.7
  27: -0.0
  28: -0.0
  29: -0.0
  30: 9.7
  31: 28.04
  32: -0.0
  33: 0.0
  34: -2.55
  35: 0.47
  36: 0.0
  37: 0.73
  38: 0.0
  39: -0.0
  40: 0.0
  41: -0.0
  42: 0.0
  43: 0.0
  44: 0.0
  45: -0.0
  46: -0.0
  47: -0.0
  48: -0.0
  49: -0.0
  50: -0.0
  51: -0.0
  52: -5.96
  53: 0.0
  54: 0.0
  55: 0.0
  56: -0.0
  57: -0.0
  58: -0.0
  59: -0.0
  60: 0.0
  61: 0.0
  62: 3.24
  63: 0.0
  64: 2.24
  65: 0.0
Model intercept: 153.74


- Regularização alta

In [33]:
alpha = 1000.0

In [34]:
model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Lasso(alpha=alpha)),
])
model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor', Lasso(alpha=1000.0))])

In [35]:
y_pred_train = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
print(f'Mean Squared Error, test, {alpha=}: {mse_train=:.2f}')

Mean Squared Error, test, alpha=1000.0: mse_train=6076.40


In [36]:
y_pred_test = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)
print(f'Mean Squared Error, test, {alpha=}: {mse_test=:.2f}')

Mean Squared Error, test, alpha=1000.0: mse_test=5361.53


In [37]:
print(f'Model coefficients:')
for i, coef in enumerate(model.named_steps['regressor'].coef_):
    print(f'  {i+1}: {coef.round(2)}')
print(f'Model intercept: {model.named_steps['regressor'].intercept_.round(2)}')

Model coefficients:
  1: 0.0
  2: 0.0
  3: 0.0
  4: 0.0
  5: 0.0
  6: 0.0
  7: -0.0
  8: 0.0
  9: 0.0
  10: 0.0
  11: 0.0
  12: 0.0
  13: 0.0
  14: 0.0
  15: 0.0
  16: 0.0
  17: -0.0
  18: 0.0
  19: 0.0
  20: 0.0
  21: 0.0
  22: 0.0
  23: 0.0
  24: 0.0
  25: 0.0
  26: -0.0
  27: 0.0
  28: 0.0
  29: 0.0
  30: 0.0
  31: 0.0
  32: 0.0
  33: 0.0
  34: -0.0
  35: 0.0
  36: 0.0
  37: 0.0
  38: 0.0
  39: 0.0
  40: 0.0
  41: -0.0
  42: 0.0
  43: 0.0
  44: 0.0
  45: 0.0
  46: 0.0
  47: -0.0
  48: 0.0
  49: 0.0
  50: 0.0
  51: 0.0
  52: -0.0
  53: 0.0
  54: 0.0
  55: 0.0
  56: -0.0
  57: 0.0
  58: -0.0
  59: -0.0
  60: 0.0
  61: 0.0
  62: 0.0
  63: 0.0
  64: 0.0
  65: 0.0
Model intercept: 153.74


### ElasticNet

#### Função de perda

A função de perda da regressão ElasticNet combina as perdas Ridge e LASSO. O hiperparâmetro $\alpha$ controla a intensidade da penalidade. O hiperparâmetro `l1_ratio` controla o balanço entre a porção LASSO e a porção Ridge.

$$
\mathcal{L} = \frac{1}{2}\text{MSE}(\overrightarrow{\theta}) + r \alpha \sum_{i=1}^{n} |\theta_i| + (1 - r) \frac{\alpha}{2} \sum_{i=1}^{n} \theta_i^2
$$

onde $r$ é `l1_ratio`.

#### Coisas importantes

- Requer normalização dos dados
- Valores mais altos de $\alpha$ servem para regularizar mais
- Valores maiores de `l1_ratio` levam o ElasticNet para um comportamento de LASSO
- Valores menores de `l1_ratio` levam o ElasticNet para um comprotamento de Ridge
- Interpretação mais difícil: a escolha de $\alpha$ e `l1_ratio` impacta os valores de $\theta$, e portanto mudam a interpretação

#### Aplicação

- Seleciona features
- Combate colinearidade
- Combate overfitting

#### Exemplo em sklearn

In [38]:
from sklearn.linear_model import ElasticNet

In [39]:
l1_ratio = 0.5

- Pouca regularização:

In [40]:
alpha = 1e-3

In [41]:
model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', ElasticNet(alpha=alpha, l1_ratio=l1_ratio)),
])
model.fit(X_train, y_train)

/home/fjayres/miniconda3/envs/ml/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.463e+05, tolerance: 2.145e+02
  model = cd_fast.enet_coordinate_descent(


Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor', ElasticNet(alpha=0.001))])

In [42]:
y_pred_train = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
print(f'Mean Squared Error, train, {alpha=}: {mse_train=:.2f}')

Mean Squared Error, train, alpha=0.001: mse_train=2514.85


In [43]:
y_pred_test = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)
print(f'Mean Squared Error, test, {alpha=}: {mse_test=:.2f}')

Mean Squared Error, test, alpha=0.001: mse_test=2645.63


In [44]:
print(f'Model coefficients:')
for i, coef in enumerate(model.named_steps['regressor'].coef_):
    print(f'  {i+1}: {coef.round(2)}')
print(f'Model intercept: {model.named_steps['regressor'].intercept_.round(2)}')

Model coefficients:
  1: -110.48
  2: -17.62
  3: -46.48
  4: 4.86
  5: -56.21
  6: 23.48
  7: -16.51
  8: 18.21
  9: -25.23
  10: -47.76
  11: 28.84
  12: 31.7
  13: -19.68
  14: 24.84
  15: -9.12
  16: -65.59
  17: 57.51
  18: 73.39
  19: 31.74
  20: 3.66
  21: -33.12
  22: 6.82
  23: 25.78
  24: 20.67
  25: -12.97
  26: -5.31
  27: -32.71
  28: -5.05
  29: 10.49
  30: 26.15
  31: 84.93
  32: -34.66
  33: 24.62
  34: -5.27
  35: -14.44
  36: 3.16
  37: 21.07
  38: -17.1
  39: 17.96
  40: 16.72
  41: -2.92
  42: -17.13
  43: 3.88
  44: -78.17
  45: -14.11
  46: 17.11
  47: 40.41
  48: -77.18
  49: 6.68
  50: 27.67
  51: -1.47
  52: -18.88
  53: 20.83
  54: 57.11
  55: -21.74
  56: -34.16
  57: -11.57
  58: -0.2
  59: 22.8
  60: 22.83
  61: -27.2
  62: 46.21
  63: 29.31
  64: 17.19
  65: 35.49
Model intercept: 153.74


- Regularização média

In [45]:
alpha = 1.0

In [46]:
model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', ElasticNet(alpha=alpha, l1_ratio=l1_ratio)),
])
model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('regressor', ElasticNet())])

In [47]:
y_pred_train = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
print(f'Mean Squared Error, test, {alpha=}: {mse_train=:.2f}')

Mean Squared Error, test, alpha=1.0: mse_train=2896.20


In [48]:
y_pred_test = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)
print(f'Mean Squared Error, test, {alpha=}: {mse_test=:.2f}')

Mean Squared Error, test, alpha=1.0: mse_test=2868.60


In [49]:
print(f'Model coefficients:')
for i, coef in enumerate(model.named_steps['regressor'].coef_):
    print(f'  {i+1}: {coef.round(2)}')
print(f'Model intercept: {model.named_steps['regressor'].intercept_.round(2)}')

Model coefficients:
  1: -0.0
  2: -1.52
  3: 4.91
  4: 2.84
  5: -0.71
  6: -0.51
  7: -1.69
  8: 0.56
  9: 4.81
  10: 0.0
  11: 0.13
  12: 0.0
  13: 2.17
  14: 1.47
  15: -0.21
  16: -0.86
  17: -1.15
  18: 0.43
  19: 1.57
  20: 0.0
  21: -1.52
  22: 0.11
  23: 0.0
  24: -1.96
  25: -2.29
  26: -3.1
  27: -0.02
  28: -0.0
  29: -0.54
  30: 5.78
  31: 5.87
  32: 1.03
  33: 0.57
  34: 0.0
  35: 2.92
  36: 5.86
  37: 3.94
  38: 3.1
  39: 0.15
  40: 0.0
  41: -0.0
  42: 1.77
  43: 4.85
  44: 1.78
  45: -0.42
  46: -0.43
  47: -1.1
  48: -0.0
  49: 0.12
  50: -0.0
  51: -0.57
  52: -2.27
  53: 0.0
  54: 0.0
  55: -0.0
  56: -0.74
  57: -1.02
  58: 0.0
  59: -1.79
  60: 0.49
  61: 1.16
  62: 0.98
  63: 4.18
  64: 2.8
  65: 0.28
Model intercept: 153.74


- Regularização alta

In [50]:
alpha = 1000.0

In [51]:
model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', ElasticNet(alpha=alpha, l1_ratio=l1_ratio)),
])
model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor', ElasticNet(alpha=1000.0))])

In [52]:
y_pred_train = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
print(f'Mean Squared Error, test, {alpha=}: {mse_train=:.2f}')

Mean Squared Error, test, alpha=1000.0: mse_train=6076.40


In [53]:
y_pred_test = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)
print(f'Mean Squared Error, test, {alpha=}: {mse_test=:.2f}')

Mean Squared Error, test, alpha=1000.0: mse_test=5361.53


In [54]:
print(f'Model coefficients:')
for i, coef in enumerate(model.named_steps['regressor'].coef_):
    print(f'  {i+1}: {coef.round(2)}')
print(f'Model intercept: {model.named_steps['regressor'].intercept_.round(2)}')

Model coefficients:
  1: 0.0
  2: 0.0
  3: 0.0
  4: 0.0
  5: 0.0
  6: 0.0
  7: -0.0
  8: 0.0
  9: 0.0
  10: 0.0
  11: 0.0
  12: 0.0
  13: 0.0
  14: 0.0
  15: 0.0
  16: 0.0
  17: -0.0
  18: 0.0
  19: 0.0
  20: 0.0
  21: 0.0
  22: 0.0
  23: 0.0
  24: 0.0
  25: 0.0
  26: -0.0
  27: 0.0
  28: 0.0
  29: 0.0
  30: 0.0
  31: 0.0
  32: 0.0
  33: 0.0
  34: -0.0
  35: 0.0
  36: 0.0
  37: 0.0
  38: 0.0
  39: 0.0
  40: 0.0
  41: -0.0
  42: 0.0
  43: 0.0
  44: 0.0
  45: 0.0
  46: 0.0
  47: -0.0
  48: 0.0
  49: 0.0
  50: 0.0
  51: 0.0
  52: -0.0
  53: 0.0
  54: 0.0
  55: 0.0
  56: -0.0
  57: 0.0
  58: -0.0
  59: -0.0
  60: 0.0
  61: 0.0
  62: 0.0
  63: 0.0
  64: 0.0
  65: 0.0
Model intercept: 153.74


### Atividade

Faça o ajuste de hiperparâmetros com validação cruzada (ou seja `GridSearchCV`) para um modelo ElasticNet para este exemplo.

## Support Vector Machines

### Modelo

A idéia central das máquinas de vetores de suporte (Support Vector Machines) é construir uma separação entre classes que seja a maior possível (a "avenida mais larga")